In [1]:
import pandas as pd

In [2]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls"

--2023-11-03 14:53:23--  http://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls
Auflösen des Hostnamens cap-payments.defra.gov.uk (cap-payments.defra.gov.uk) … 40.127.153.231
Verbindungsaufbau zu cap-payments.defra.gov.uk (cap-payments.defra.gov.uk)|40.127.153.231|:80 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 307 Temporary Redirect
Platz: https://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls [folgend]
--2023-11-03 14:53:23--  https://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls
Verbindungsaufbau zu cap-payments.defra.gov.uk (cap-payments.defra.gov.uk)|40.127.153.231|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 33646592 (32M) [application/vnd.ms-excel]
Wird in ‘2014_All_CAP_Search_Results_Data_P14.xls’ gespeichert.

2014_All_CAP_Search 100%[===================>]  32,09M   884KB/s    in 47s     

2023-11-03 14:54:10 (699 KB/s) - ‘201

In [1]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls"

--2016-06-02 12:01:45--  http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls
Resolving cap-payments.defra.gov.uk... 194.1.210.50
Connecting to cap-payments.defra.gov.uk|194.1.210.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33282560 (32M) [application/vnd.ms-excel]
Saving to: '2015_All_CAP_Search_Results_Data_P14.xls'

2015_All_CAP_Search 100%[===================>]  31.74M  1.91MB/s    in 12s     

2016-06-02 12:01:57 (2.72 MB/s) - '2015_All_CAP_Search_Results_Data_P14.xls' saved [33282560/33282560]



In [2]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2016_All_CAP_Search_Results_Data_P14.xlsx"

--2018-02-05 11:36:33--  http://cap-payments.defra.gov.uk/Download/2016_All_CAP_Search_Results_Data_P14.xlsx
Resolving cap-payments.defra.gov.uk... 194.1.210.50
Connecting to cap-payments.defra.gov.uk|194.1.210.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15767150 (15M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘2016_All_CAP_Search_Results_Data_P14.xlsx’

2016_All_CAP_Search 100%[===================>]  15.04M  2.35MB/s    in 7.7s    

2018-02-05 11:36:41 (1.96 MB/s) - ‘2016_All_CAP_Search_Results_Data_P14.xlsx’ saved [15767150/15767150]



In [3]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2022_All_CAP_Search_Results_Data_P14.xlsx"

--2023-11-03 14:54:12--  http://cap-payments.defra.gov.uk/Download/2022_All_CAP_Search_Results_Data_P14.xlsx
Auflösen des Hostnamens cap-payments.defra.gov.uk (cap-payments.defra.gov.uk) … 40.127.153.231
Verbindungsaufbau zu cap-payments.defra.gov.uk (cap-payments.defra.gov.uk)|40.127.153.231|:80 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 307 Temporary Redirect
Platz: https://cap-payments.defra.gov.uk/Download/2022_All_CAP_Search_Results_Data_P14.xlsx [folgend]
--2023-11-03 14:54:12--  https://cap-payments.defra.gov.uk/Download/2022_All_CAP_Search_Results_Data_P14.xlsx
Verbindungsaufbau zu cap-payments.defra.gov.uk (cap-payments.defra.gov.uk)|40.127.153.231|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 11031657 (11M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Wird in ‘2022_All_CAP_Search_Results_Data_P14.xlsx’ gespeichert.

2022_All_CAP_Search 100%[===================>]  10,52M  1,27MB/s    in 9,

In [4]:
YEAR = 2022

In [6]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 195.8 kB/s eta 0:00:000:00:010:00:01:01


In [7]:
excel_file = pd.ExcelFile('%s_All_CAP_Search_Results_Data_P14.xlsx' % YEAR)
excel_file.sheet_names

['DAERA', 'SGRPID', 'WG', 'RPA', 'RPA 2']

In [8]:
id_vars = ['Year', 'BeneficiaryCode', 'BeneficiaryName_F201',
       'PostcodePrefix_F202B', 'TownCity_F202C',
#        'OtherEAGFTotal', 'DirectEAGFTotal', 'RuralDevelopmentTotal', 'Total',
        'PayingAgencyLink']


def get_melted_df(excel_file, sheet_name):
    df_raw = excel_file.parse(sheet_name)
    df_raw = df_raw[df_raw['Year'].notnull()]
    all_columns = list(df_raw.columns)
    value_vars = list(set(all_columns) - set(id_vars))
    df_raw = df_raw.rename(columns={x: x.strip() for x in value_vars})
    value_vars = [v.strip() for v in value_vars]
    df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, var_name='scheme', value_name='amount')
    df = df[df['amount'].notnull()]
    return df

df = pd.concat([get_melted_df(excel_file, sheet) for sheet in excel_file.sheet_names])
df.head()

,Year,BeneficiaryCode,BeneficiaryName_F201,PostcodePrefix_F202B,TownCity_F202C,PayingAgencyLink,scheme,amount
1058,2022,NaN,BRITISH TRUST FOR ORNITHOLOGY,IP24,ENGLAND,DAERA,Technical assist,91631.32
1300,2022,NaN,COUNTRYSIDE SERVICES LIMITED,BT71,CO TYRONE,DAERA,Technical assist,225512.40
1398,2022,NaN,DARD FARM POLICY BRANCH,BT4,BELFAST CO DOWN,DAERA,Technical assist,2904812.46
195652,2022,NaN,AGHYARAN GAA CLUB,BT81,CO.TYRONE,DAERA,Support for Leader local development,30725.22
195769,2022,NaN,ANTRIM & NEWTOWNABBEY BOROUGH COUNCIL,BT36,CO.ANTRIM,DAERA,Support for Leader local development,188992.88


In [10]:
df = df.rename(columns={
    'Year': 'year',
    'BeneficiaryCode': 'recipient_id',
    'BeneficiaryName_F201': 'recipient_name',
    'PostcodePrefix_F202B': 'recipient_postcode',
    'TownCity_F202C': 'recipient_location',
    'PayingAgencyLink': 'agency',
})
df['country'] = 'GB'
df['currency'] = 'GBP'
df['year'] = df['year'].astype('int')
df['recipient_name'] = df['recipient_name'].str.strip()
df['recipient_postcode'] = df['recipient_postcode'].str.strip()
df['recipient_location'] = df['recipient_location'].str.strip()
df.loc[df['recipient_id'].notnull(), 'recipient_name'] = None
#df.set_value(df['recipient_id'].notnull(), 'recipient_name', None)
df.head()

,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
1058,2022,NaN,BRITISH TRUST FOR ORNITHOLOGY,IP24,ENGLAND,DAERA,Technical assist,91631.32,GB,GBP
1300,2022,NaN,COUNTRYSIDE SERVICES LIMITED,BT71,CO TYRONE,DAERA,Technical assist,225512.40,GB,GBP
1398,2022,NaN,DARD FARM POLICY BRANCH,BT4,BELFAST CO DOWN,DAERA,Technical assist,2904812.46,GB,GBP
195652,2022,NaN,AGHYARAN GAA CLUB,BT81,CO.TYRONE,DAERA,Support for Leader local development,30725.22,GB,GBP
195769,2022,NaN,ANTRIM & NEWTOWNABBEY BOROUGH COUNCIL,BT36,CO.ANTRIM,DAERA,Support for Leader local development,188992.88,GB,GBP


In [11]:
def set_recipient_id(row):
    if pd.isnull(row['recipient_id']):
        row['recipient_id'] = 'GB-%s-%s' % (row['recipient_postcode'], row['recipient_name'])
    else:
        row['recipient_id'] = 'GB-%s' % int(row['recipient_id'])
    return row

df = df.apply(set_recipient_id, axis=1)
df.head()

,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
1058,2022,GB-IP24-BRITISH TRUST FOR ORNITHOLOGY,BRITISH TRUST FOR ORNITHOLOGY,IP24,ENGLAND,DAERA,Technical assist,91631.32,GB,GBP
1300,2022,GB-BT71-COUNTRYSIDE SERVICES LIMITED,COUNTRYSIDE SERVICES LIMITED,BT71,CO TYRONE,DAERA,Technical assist,225512.40,GB,GBP
1398,2022,GB-BT4-DARD FARM POLICY BRANCH,DARD FARM POLICY BRANCH,BT4,BELFAST CO DOWN,DAERA,Technical assist,2904812.46,GB,GBP
195652,2022,GB-BT81-AGHYARAN GAA CLUB,AGHYARAN GAA CLUB,BT81,CO.TYRONE,DAERA,Support for Leader local development,30725.22,GB,GBP
195769,2022,GB-BT36-ANTRIM & NEWTOWNABBEY BOROUGH COUNCIL,ANTRIM & NEWTOWNABBEY BOROUGH COUNCIL,BT36,CO.ANTRIM,DAERA,Support for Leader local development,188992.88,GB,GBP


In [12]:
len(df)

846114

In [13]:
df = df[~df['scheme'].isin([x for x in df['scheme'].value_counts().index if x.endswith('Total')])]
df['scheme'].value_counts()

scheme
Basic payment scheme - UK funded                                                    143841
Agri-environment-climate                                                             26305
Greening - UK payments - UK funded                                                   18848
Redistributive payment - UK funded                                                   16313
Payments to areas facing natural constraints - UK funded                             10713
Other direct aids - UK funded                                                         7293
Investment in forest area development and viability                                   6306
Investments in physical assets                                                        4751
Young farmers - UK funded                                                             4277
Investments in physical assets - UK funded                                            3518
Organic farming                                                                    

In [14]:
df.to_csv('gb_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')